In [1]:
!nvidia-smi

Tue Apr  4 16:56:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.36       Driver Version: 512.36       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   36C    P8     6W /  N/A |    488MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn
torch.device('cpu'),torch.device('cuda'),torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [5]:
def try_gpu(i=0):
    if torch.cuda.device_count()>=i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpu():
    devices=[torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]
try_gpu(),try_gpu(10),try_all_gpu()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [6]:
x=torch.tensor([1,2])
#查询张量所在的设备。 默认情况下，张量是在CPU上创建的。
x.device

device(type='cpu')

In [8]:
#存储在GPU上
x=torch.arange(6,device=try_gpu())
x

tensor([0, 1, 2, 3, 4, 5], device='cuda:0')

In [9]:
#假设我们至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量。
Y=torch.rand(2,3,device=try_gpu(1))
Y

tensor([[0.5113, 0.5027, 0.9094],
        [0.4770, 0.0885, 0.5112]])

In [10]:
#神经网络模型可以指定设备。 下面的代码将模型参数放在GPU上
net=nn.Sequential(nn.Linear(10,2))
net=net.to(device=try_gpu())

In [14]:
x=torch.rand(2,10,device=try_gpu())  #进行运算的数据和模型必须在同一个设备上不然会报错
net(x)

tensor([[-0.1317, -0.2263],
        [-0.3535, -0.1978]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [15]:
net[0].weight.data.device

device(type='cuda', index=0)

In [18]:
net[0].weight.data

tensor([[-0.1700, -0.1785, -0.1546,  0.1293, -0.2791,  0.1768,  0.1247, -0.0695,
          0.0887,  0.0085],
        [ 0.0094, -0.1382,  0.2150,  0.1782, -0.2470,  0.0091, -0.2387, -0.2104,
         -0.0872,  0.0108]], device='cuda:0')